# Validation Examples
#### Frederik Kelbel, Imperial College London

## Dependencies

In [1]:
import torch
import plotly.graph_objects as go
import numpy as np
from operators import div, Δ, D, mdotb, bdotm, mdotm, bdotb
from DGM import DeepPDESolver
from pdes import PDE
from scipy.integrate import quad
from plotly.subplots import make_subplots
from configs import CONFIG_PARABOLIC_PDES as MODEL_CONFIG

## Plotting

In [2]:
def plot_losses(losses, avg_over=10):
    avgs = np.convolve(losses, np.ones(avg_over), 'valid') / avg_over
    fig = make_subplots(rows=1, cols=1)
    fig.add_trace(go.Scatter(x=np.arange(len(avgs)), y=avgs, mode='lines', name="Error at x=0.1"), row=1, col=1)
    fig.update_layout(
        title="Loss",
        xaxis_title="Iterations",
        yaxis_title="Loss",
        font=dict(
            family="Courier New, monospace",
            size=14
        )
    )
    fig.show()
    
def plot_1D_functions(solver, sol):
    fig = make_subplots(rows=1, cols=2, 
                   specs=[[{'type': 'surface'}, {'type': 'surface'}]])
    xs = np.linspace(0, 1, 100)
    ts = np.linspace(0.01, 1, 100)
    us_pred = np.array([[solver.u(x, t).item() for x in xs] for t in ts])
    us = np.array([[sol(x, t) for x in xs] for t in ts])
    x_mesh, t_mesh = np.meshgrid(xs, ts)
    fig.add_trace(go.Surface(z=us, showscale=False), row=1, col=1)
    fig.add_trace(go.Surface(z=us_pred), row=1, col=2)
    fig.update_layout(title='Solution | Approximation',
                  scene = dict(
                    xaxis_title="t",
                    yaxis_title="x",
                    zaxis_title="u(x, t)"),
                  scene2 = dict(
                    xaxis_title="t",
                    yaxis_title="x",
                    zaxis_title="u(x, t)"),
                  margin=dict(l=50, r=50, b=50, t=50))
    fig.show()

## Partial Differential Equations

### Poisson Equation:
$$
\begin{cases}
-\Delta u = f \quad & \text{in } [0, 1]^2\\
u = g \quad & \text{on } \partial[0, 1]^2
\end{cases}
$$

In [20]:
class POISSON(PDE):
    def __init__(self):
        super().__init__()
        self.f = lambda var: 2#(16*np.pi**2*(var[1]-1)**2*var[1]**2-2*(var[1]-1)**2-8*(var[1]-1)*var[1]-2*var[1]**2)*torch.sin(4*np.pi*var[0])
        self.g = lambda var: 0
        boundary_batch = 128
        def boundary(var):
            k = int(boundary_batch/4)
            var[0][:k] = 1
            var[0][k:2*k] = -1
            var[1][2*k:3*k] = 1
            var[1][3*k:] = -1
            
            var[0][0:4] = 1
            var[1][0:4] = 1
            var[0][k:k+2] = -1
            var[1][k:k+2] = 1
            var[0][2*k:2*k+2] = 1
            var[1][2*k:2*k+2] = -1
            var[0][3*k:3*k+2] = -1
            var[1][3*k:3*k+2] = -1
            return var
            
        self.var_dim = 2 # var = (x, t)
        self.equation = lambda u, var: (1/2)*torch.sum(D(u, var)**2, dim=-1, keepdim=True) -u*self.f(var)
        #self.equation = lambda u, var: -Δ(u, var) -self.f(var)
        self.domain_func = [(lambda var: [(var[0]-0.5)*2, (var[1]-0.5)*2], 128)]
        self.boundary_cond = [lambda u, var: u -self.g(var)]
        self.boundary_func = [(lambda var: boundary(var), boundary_batch)]
       

In [21]:
 POiSSON_MODEL_CONFIG = {
    "hidden_dim": 32,
    "learning_rate": 5e-3,
    "loss_weights": (1, 500),
    "sampling_method": "uniform",
    "lr_decay": 0.99,
    "network_type": "FF",
    "optimiser": "Adam",
     "method": "Ritz"
}
eq = POISSON()
model = POiSSON_MODEL_CONFIG
solver = DeepPDESolver(model, eq)
losses = list(solver.train(1000))
plot_losses(losses)

100%|██████████| 1000/1000 [00:03<00:00, 327.69 it/s]


In [22]:
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'surface'}]])
xs = np.linspace(-1, 1, 100)
ys = np.linspace(-1, 1, 100)
us_pred = np.array([[solver.u(x, y).item() for x in xs] for y in ys])
us = us_pred #np.array([[sol(x, t) for x in xs] for t in ts])
x_mesh, y_mesh = np.meshgrid(xs, ys)
fig.add_trace(go.Surface(x=ys, y=xs, z=us, showscale=False), row=1, col=1)
#fig.add_trace(go.Surface(x=ys, y=xs, z=us_pred), row=1, col=2)
fig.update_layout(title='Solution | Approximation',
                  scene = dict(
                    xaxis_title="y",
                    yaxis_title="x",
                    zaxis_title="u(x, t)"),
                  scene2 = dict(
                    xaxis_title="y",
                    yaxis_title="x",
                    zaxis_title="u(x, t)"),
                  margin=dict(l=50, r=50, b=50, t=50))
fig.show()

### Nonhomogeneous Transport:
$$
\begin{cases}
u_t + b (\nabla \cdot u) = f \quad & \text{in } [0, 1]^\texttt{x_dim=1} \times [0, 1]\\
u = g \quad & \text{on } [0, 1]^\texttt{x_dim=1} \times {t=0}
\end{cases}
$$

In [ ]:
class TRANSPORT(PDE):
    def __init__(self):
        super().__init__()
        self.b = 0.3
        self.f = lambda x, t: 2*t + x*t
        self.g = lambda x: x
        self.var_dim = 2 # var = (x, t)
        self.equation = lambda u, var: div(u, var[1]) + self.b * div(u, var[0]) - self.f(var[0], var[1])
        self.domain_func = [(lambda var: var, 64)]
        self.boundary_cond = [lambda u, var: u -self.g(var[0])]
        self.boundary_func = [(lambda var: [var[0], var[1]*0], 64)]
        

In [ ]:
eq = TRANSPORT()
model = MODEL_CONFIG
solver = DGMSolver(model, eq)
losses = list(solver.train(200))
plot_losses(losses)

In [ ]:
if eq.var_dim == 2:
    f = lambda s, x, t, b: eq.f(x+(s-t)*b, s)
    Transport_sol =\
        lambda x, t: eq.g(x-eq.b*t) + quad(f, 0, t, args=(x, t, eq.b))[0]
    plot_1D_functions(solver, Transport_sol)

### Inviscid Burger's Equation:
$$
\begin{cases}
u_t + u (\nabla \cdot u) = 0 \quad & \text{in } [0, 1]^\texttt{x_dim=1} \times [0, 1]\\
u = g \quad & \text{on } [0, 1]^\texttt{x_dim=1} \times {t=0}
\end{cases}
$$

In [ ]:
class INVISCID_BURGERS(PDE):
    def __init__(self):
        super().__init__()
        self.var_dim = 2 # var = (x, t)
        self.equation = lambda u, var: div(u, var[1]) + u * div(u, var[0])
        self.domain_func = [(lambda var: var, 64)]
        self.boundary_cond = [lambda u, var: u - var[0]]
        self.boundary_func = [(lambda var: [var[0], var[1]*0], 64)]

In [ ]:
eq = INVISCID_BURGERS()
model = MODEL_CONFIG
solver = DGMSolver(model, eq)
losses = list(solver.train(300))
plot_losses(losses)

In [ ]:
if eq.var_dim==2:
    INVISCID_BURGERS_sol = lambda x, t: x / (1 + t)
    plot_1D_functions(solver, INVISCID_BURGERS_sol)

### Diffusion Advection Equation:
$$
\begin{cases}
u_t + a u_x - b \Delta u = 0 \quad & \text{in } [0, 1]^{\texttt{x_dim}=1} \times [0, 1]\\
u = \exp\Big( -\frac{(x-1)^2}{b}\Big) \quad & \text{on } [0, 1]^{\texttt{x_dim}=1} \times {t=0} \\
u = \frac{1}{\sqrt{4t+1}} \exp\Big(-\frac{(x-1-at)^2}{b(4t+1)} \Big) \quad & \text{on } \{0, 1\}^{\texttt{x_dim}=1}\times [0, 1]
\end{cases}
$$

In [ ]:
class HEAT(PDE):
    def __init__(self):
        super().__init__()
        self.a = 0.01
        self.b = 10
        self.var_dim = 2
        self.equation = lambda u, var: div(u, var[1]) + self.a * div(u, var[0]) - self.b * Δ(u, var[0])
        self.domain_func = [(lambda var: var, 64)]
        self.boundary_cond = [lambda u, var: u - 1/torch.sqrt(4*var[1]+1)*torch.exp(-(var[0]-1-self.a*var[0])**2/(self.b*(4*var[1]+1))),
                             lambda u, var: u - torch.exp(-(var[0]-1)**2/self.b)]
        self.boundary_func = [(lambda var: [torch.where(var[0] > 0.5, 1.0, 0.0), var[1]], 16),
                             (lambda var: [var[0], var[1]*0], 16)]

In [ ]:
HEAT_MODEL_CONFIG = {
    "hidden_dim": 128,
    "learning_rate": 5e-3,
    "loss_weights": (1, 3),
    "sampling_method": "uniform",
    "lr_decay": 0.98,
    "network_type": "GRU",
    "optimiser": "Adam"
}
eq = HEAT()
model = HEAT_MODEL_CONFIG
solver = DGMSolver(model, eq)
losses = list(solver.train(600))
plot_losses(losses)

In [ ]:
if eq.var_dim == 2:
    HEAT_sol = lambda x, t: 1/np.sqrt(4*t+1)*np.exp(-(x-1-eq.a*t)**2/(eq.b*(4*t+1)))
    plot_1D_functions(solver, HEAT_sol)

### Transport Equation in 2D:
$$
\begin{cases}
u_t + y u_x - x u_y = 0 \quad & \text{in } [0, 1]^\texttt{2} \times [0, 1]\\
u = \exp(-(x-1)^2 -(y-1)^2) \quad & \text{on } [0, 1]^\texttt{2} \times {t=0}
\end{cases}
$$

In [ ]:
class TRANSPORT2D(PDE):
    def __init__(self):
        super().__init__()
        self.var_dim = 3 # var = (x, y, t)
        self.equation = lambda u, var: div(u, var[2]) + var[1]*div(u, var[0]) - var[0]*div(u, var[1]) 
        self.domain_func = [(lambda var: var, 128)]
        self.boundary_cond = [lambda u, var: u -torch.exp(-(var[0]-1)**2-(var[1]-1)**2)]
        self.boundary_func = [(lambda var: var[0:2] + [var[1]*0], 32)]
      

In [ ]:
TRANSPORT2D_MODEL_CONFIG = {
    "hidden_dim": 64,
    "learning_rate": 5e-3,
    "loss_weights": (2, 1),
    "sampling_method": "uniform",
    "lr_decay": 0.98,
    "network_type": "GRU",
    "optimiser": "Adam"
}
eq = TRANSPORT2D()
model = TRANSPORT2D_MODEL_CONFIG
solver = DGMSolver(model, eq)
losses = list(solver.train(900))
plot_losses(losses)

In [ ]:
TRANSPORT2D_sol = lambda x, y, t: np.exp(-(x**2+y**2)+2*(x+y)*np.cos(t)+2*(x-y)*np.sin(t)-2)
domain_error = np.sum([[[np.abs(solver.u(x, y, t).item()-TRANSPORT2D_sol(x, y, t)) for x in np.linspace(0, 1, 50)] for y in np.linspace(0, 1, 50)] for t in np.linspace(0.01, 1,10)], axis=0)

In [ ]:
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'heatmap'}]])
fig.add_trace(go.Heatmap(z=domain_error, coloraxis = "coloraxis", zsmooth='best'), row=1, col=1)
fig.update_layout(title='Error density',
                  coloraxis = {'colorscale':'Viridis'},
                  xaxis_title="x",
                  yaxis_title="y",
                  margin=dict(l=20, r=20, b=20, t=50))
fig.show()

### Riccati-Equation, LQR-Problem:
$$
\begin{cases}
\nabla J(x)^T f(x)-\frac{1}{4} \nabla J(x)^T B(x) D^{-1} B(x)^T \nabla J(x) + x^T C x = 0
\end{cases}
$$

In [ ]:
var_dim = 10
C = torch.rand(10, 10)
C = torch.mm(C, C.T)
C+= torch.eye(10)

In [ ]:
class LQR(PDE):
    def __init__(self):
        super().__init__()
        self.f = lambda var: torch.cat(var, dim=-1) + 1
        self.B = torch.eye(var_dim)*2
        self.C = C
        self.D = torch.eye(var_dim)*0.2
        self.var_dim = var_dim
        self.equation = lambda J, var: (D(J, var) |bdotb| self.f(var)) - (((((1/4)* D(J, var) |bdotm| self.B) |bdotm| torch.inverse(self.D))
                                                                         |bdotm| self.B.T) |bdotb| D(J, var)) + ((var |bdotm| self.C) |bdotb| var) 
        self.domain_func = [(lambda var: var, 128)]

In [ ]:
CONFIG_LQR = {
    "hidden_dim": 256,
    "learning_rate": 5e-3,
    "loss_weights": (1, 0),
    "lr_decay": 0.99,
    "sampling_method": "uniform",
    "network_type": "GRU",
    "optimiser": "Adam"
}
eq = LQR()
model = CONFIG_LQR
solver = DGMSolver(model, eq)
losses = list(solver.train(800))
plot_losses(losses)

Algebraic Riccati Equation grows to quickly in dimensions. For 10-dim control output, we solve a 100-dim equation.

### Fokker-Planck Equation for the OU Process:
$$
\begin{cases}
\partial_t p + k p + k(x-\mu) \partial_x p - \frac{1}{2} \sigma^2 \partial_{xx} p = 0 \quad [-4, 4] \times [0, 1] \\
p(0, x) = \delta(x - x_0) \approx \lim_{a \rightarrow 0}\frac{1}{|a| \sqrt{\pi}} e^{-(\frac{x-x_0}{a})^2} \quad & \text{on } [-4, 4] \times {t=0} \\
\end{cases}
$$

Notes: Exponentiating assumes to much knowledge. PIADGM paper is cheating. GRUNetwork performs better for some reason.

In [ ]:
class OUPROCESS(PDE):
    def __init__(self):
        super().__init__()
        self.mu = 0.5
        self.sigma = 2.0
        self.k = 0.0
        a = 0.1
        self.var_dim = 2 # var = (x, t)   
        self.equation = lambda p, var: div(p, var[1]) + self.k*p + self.k*(var[0]-self.mu)*div(p, var[0])-(1/2)*self.sigma**2*Δ(p, var[0])
        self.domain_func = [(lambda var: [8*var[0]-4, var[1]], 64)]
        self.boundary_cond = [lambda p, var: p -(1/np.sqrt(2*np.pi*a))*torch.exp(-(var[0]**2/(2*a)))]
        self.boundary_func = [(lambda var: [8*var[0]-4, 0*var[1]], 64)]

In [ ]:
CONFIG_OU = {
    "hidden_dim": 64,
    "learning_rate": 5e-3,
    "loss_weights": (1.5, 1),
    "lr_decay": 0.99,
    "sampling_method": "uniform",
    "network_type": "DENS",
    "optimiser": "Adam"
}
eq = OUPROCESS()
model = CONFIG_OU
solver = DGMSolver(model, eq)
losses = list(solver.train(1000))
plot_losses(losses)

In [ ]:
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'surface'}]])
xs = np.linspace(-4, 4, 100)
ts = np.linspace(0.01, 1, 100)
us_pred = np.array([[solver.u(x, t).item() for x in xs] for t in ts])
norms = [np.trapz(us, xs) for us in us_pred]
us_pred = np.array([ us/n for us, n in zip(us_pred, norms)])
fig.add_trace(go.Surface(x=xs, y=ts, z=us_pred), row=1, col=1)
fig.update_layout(title='Solution | Approximation',
                  scene = dict(
                      xaxis_title="x",
                      yaxis_title="t",
                      zaxis_title="p(x, t)"),
                  margin=dict(l=50, r=50, b=50, t=50))
fig.show()

### Stationary Fokker-Planck Equation for the OU Process (Smoluchowski equation) N-dimensional:
$$
\begin{cases}
\nabla \cdot (\nabla V p_s) + \beta^{-1} \Delta p = 0 \quad [-4, 4] \times [0, 1] \\
p(0, x) = \delta(x - x_0) \approx \lim_{a \rightarrow 0}\frac{1}{|a| \sqrt{\pi}} e^{-(\frac{x-x_0}{a})^2} \quad & \text{on } [-4, 4] \times {t=0} \\
\end{cases}
$$

In [ ]:
class STATOUPROCESS(PDE):
    def __init__(self):
        super().__init__()
        self.var_dim = 10
        random_M = torch.rand(self.var_dim, self.var_dim)
        self.β = 2.0
        self.V = lambda var: (var |bdotm| random_M) |bdotb| var
        a = 0.1
        
        self.equation = lambda p, var: (1/self.β)*Δ(p, var) + Δ(self.V(var), var)*p + (D(self.V(var), var) |bdotb| D(p, var))
        self.domain_func = [(lambda var: var, 128)]

In [ ]:
CONFIG_STATOU = {
    "hidden_dim": 64,
    "learning_rate": 5e-3,
    "loss_weights": (1.5, 1),
    "lr_decay": 0.99,
    "sampling_method": "uniform",
    "network_type": "DENS",
    "optimiser": "Adam"
}
eq = STATOUPROCESS()
model = CONFIG_STATOU
solver = DGMSolver(model, eq)
losses = list(solver.train(1000))
plot_losses(losses)